In [96]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [97]:
body = {
    "settings": {
        "analysis": {
            "analyzer": {
                "analyzer_index": {
                    "tokenizer": "ik_max_word",
                    "char_filter": ["emoticons"],
                    "filter": ["english_stop"]
                },
                "analyzer_search": {
                    "tokenizer": "ik_max_word",
                    "char_filter": ["emoticons"],
                    "filter": [
                        "my_synonyms",  # 推荐搜索时使用
                        "english_stop"
                    ]
                }
            },
            "filter": {
                "my_synonyms": {
                    "type": "synonym",
                    "synonyms": [
                        "母亲 , 妈妈 , mother",
                        "父亲 , 爸爸 , father",
                        "老虎 , 狮子 => 动物",
                        "西红柿 , 黄瓜, 香蕉 => 水果, 蔬菜"
                    ],
                    "updateable": True
                },
                "english_stop": {
                    "type": "stop",
                    "stopwords": "_english_"
                }
            },
            "char_filter": {
                "emoticons": {
                    "type": "mapping",
                    "mappings": [
                        "II => 2",
                        "ii => 2",
                        "Ⅱ => 2"
                        "III => 3",
                        "iii => 3",
                        "Ⅲ => 3"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "information": {
                "type": "text",
                "analyzer": "analyzer_index"
            }
        }
    }
}

if es.indices.exists('es0'):
    es.indices.delete('es0')
es.indices.create('es0', body=body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [98]:
es.index(index='es0', id=0, body={"information": "我爱妈妈"})
es.index(index='es0', id=1, body={"information": "我爱mother"})
es.index(index='es0', id=2, body={"information": "我爱father"})
es.index(index='es0', id=3, body={"information": "动物很可爱"})
es.index(index='es0', id=4, body={"information": "小孩很可爱"})
es.index(index='es0', id=5, body={"information": "蔬菜有利于健康"})
es.index(index='es0', id=6, body={"information": "香蕉很好吃"})
es.index(index='es0', id=7, body={"information": "水果很好吃"})

{'_index': 'es0',
 '_type': '_doc',
 '_id': '7',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 7,
 '_primary_term': 1}

In [99]:
es.search(index='es0', body={
    "query": {
        "match": {
            "information": {
                "query": "我爱我的母亲",
                # 使用自定义analyzer(默认使用索引时该字段analyzer,本例中为analyzer_index)
                "analyzer": "analyzer_search"  # ★★★★★仅full_text_queries可以指定该字段
            }
        }
    }
})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 2.8251233,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 2.8251233,
    '_source': {'information': '我爱妈妈'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '1',
    '_score': 2.8251233,
    '_source': {'information': '我爱mother'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '2',
    '_score': 0.9751481,
    '_source': {'information': '我爱father'}}]}}

In [100]:
es.search(index='es0', body={
    "query": {
        "match": {
            "information": {
                "query": "老虎很可爱",
                "analyzer": "analyzer_search" 
            }
        }
    }
})

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 4.4950805,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '3',
    '_score': 4.4950805,
    '_source': {'information': '动物很可爱'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '4',
    '_score': 2.6451051,
    '_source': {'information': '小孩很可爱'}}]}}

In [101]:
es.search(index='es0', body={
    "query": {
        "match": {
            "information": {
                "query": "香蕉很好吃",  # 香蕉被转换为水果(单向同义词),故水果很好吃_score更高
                "analyzer": "analyzer_search" 
            }
        }
    }
})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 4.4950805,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '7',
    '_score': 4.4950805,
    '_source': {'information': '水果很好吃'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '6',
    '_score': 2.6451051,
    '_source': {'information': '香蕉很好吃'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '5',
    '_score': 1.4683185,
    '_source': {'information': '蔬菜有利于健康'}}]}}